In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from datasets import load_dataset
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
from lm_polygraph.estimators import *
from lm_polygraph.utils.model import WhiteboxModel
from lm_polygraph.utils.dataset import Dataset
from lm_polygraph.utils.processor import Logger
from lm_polygraph.utils.manager import UEManager
from lm_polygraph.ue_metrics import PredictionRejectionArea
from lm_polygraph.generation_metrics import RougeMetric, BartScoreSeqMetric, ModelScoreSeqMetric, ModelScoreTokenwiseMetric
from lm_polygraph.utils.builder_enviroment_stat_calculator import (
    BuilderEnvironmentStatCalculator
)
from lm_polygraph.defaults.register_default_stat_calculators import (
    register_default_stat_calculators,
)
from lm_polygraph.utils.factory_stat_calculator import StatCalculatorContainer
from omegaconf import OmegaConf

# Specify HyperParameters

In [ ]:
model_path = "facebook/wmt19-en-de"
device = "cuda"
model_type = "Whitebox"
dataset_name = ("wmt14", "de-en")
batch_size = 4
seed = 42

# Initialize Model

In [ ]:
base_model = FSMTForConditionalGeneration.from_pretrained(
    model_path,
    device_map=device,
)
tokenizer = FSMTTokenizer.from_pretrained(model_path)

model = WhiteboxModel(base_model, tokenizer, model_type="Seq2SeqLM")

# Train and Eval Datasets

In [ ]:
hf_dataset = load_dataset(*dataset_name)

dataset = Dataset(
    x=[txt["en"] for txt in hf_dataset["test"]["translation"]],
    y=[txt["de"] for txt in hf_dataset["test"]["translation"]],
    batch_size=batch_size,
)
dataset.subsample(16, seed=seed)

train_dataset = Dataset(
    x=[txt["en"] for txt in hf_dataset["train"]["translation"][:1000]],
    y=[txt["de"] for txt in hf_dataset["train"]["translation"][:1000]],
    batch_size=batch_size,
)
train_dataset.subsample(16, seed=seed)

# Metric, UE Metric, and UE Methods

In [ ]:
ue_methods = [MaximumSequenceProbability(), 
              SemanticEntropy(),
              MahalanobisDistanceSeq("encoder"),]

ue_metrics = [PredictionRejectionArea()]

metrics = [RougeMetric('rougeL')]

loggers = [Logger()] 

# Stat Calculators

In [ ]:
TrainingStatistic_config = {
    "dataset": dataset_name,
    "text_column": 'en',
    "label_column": 'de',
    "description": '',
    "prompt": "",
    "few_shot_split": 'train',
    "train_split": 'train',
    "load_from_disk": False,
    "subsample_train_dataset": 10,
    "n_shot": 5,
    "train_dataset": dataset_name,
    "train_test_split": False,
    "background_train_dataset": 'allenai/c4',
    "background_train_dataset_text_column": 'text',
    "background_train_dataset_label_column": 'url',
    "background_train_dataset_data_files": 'en/c4-train.00000-of-01024.json.gz',
    "background_load_from_disk": False,
    "subsample_background_train_dataset": 10,
    "batch_size": 1,
    "seed": 1
}

In [ ]:
# register default stat calculators
result_stat_calculators = dict()
scs = register_default_stat_calculators(model_type)
for sc in scs:
    result_stat_calculators[sc.name] = sc

# register TrainingStatisticExtractionCalculator for the Mahalanobis Distance method
result_stat_calculators.update(
    {
        "TrainingStatisticExtractionCalculator": StatCalculatorContainer(
            name="TrainingStatisticExtractionCalculator",
            cfg=OmegaConf.create(TrainingStatistic_config),
            stats=["train_embeddings", "background_train_embeddings", "train_greedy_log_likelihoods"],
            dependencies=[],
            builder="lm_polygraph.defaults.stat_calculator_builders.default_TrainingStatisticExtractionCalculator",
        )
    }
)
    
builder_env_stat_calc = BuilderEnvironmentStatCalculator(model=model)
available_stat_calculators = list(result_stat_calculators.values())

# Initialize UE Manager

In [ ]:
man = UEManager(
    data=dataset,
    model=model,
    estimators=ue_methods,
    builder_env_stat_calc=builder_env_stat_calc,
    available_stat_calculators=available_stat_calculators,
    generation_metrics=metrics,
    ue_metrics=ue_metrics,
    processors=loggers,
    ignore_exceptions=False,
    max_new_tokens=100
)

# Compute Results

In [ ]:
results = man()

In [ ]:
for key in results.keys():
    print(f"UE Score: {key[1]}, Metric: {key[2]}, UE Metric: {key[3]}, Score: {results[key]:.3f}")